# Materiały do zadań - grafy

## Podstawowa funkcjonalność biblioteki networkx

Poniższy materiały opisują tylko fragment możliwości biblioteki. Pełna dokumentacja znajduje się pod adresem: [[LINK]](https://networkx.org/documentation/stable/_downloads/networkx_reference.pdf), gdzie można znaleźć np. inne metody wczytywania grafów, algorytmy, narzędzia itp.

In [ ]:
import networkx as nx

### Tworzenie grafu

Graf możemy utworzyć korzystając z klasy `nx.Graph`

In [ ]:
graph = nx.Graph()

Następnie możemy dodać krawędzie do grafu za pomocą metody `add_edge`

In [ ]:
graph.add_edge(1, 2)
graph.add_edge(1, 3)
graph.add_edge(2, 3)
graph.add_edge(1, 4)
graph.add_edge(2, 4)

Możemy też dodać samego wierzchołka zapomocą metody `add_node`

In [ ]:
graph.add_node(5)

W takim ustawieniu mamy graf, które posiada wierzchołki izolowane (nie posiadające krawędzi). W celu sprawdzenia czy w grafie mamy izolowane wierzchołki możemy wykorzystać metodę `isolates()` z modułu `networkx`

In [ ]:
list(nx.isolates(graph))

Listę krawędzi możemy wyświetlić za pomocy metody `edges`

In [ ]:
graph.edges()

Warto zauważyć, że zwracany obiekt nie jest pythonową listą, lecz obiektem `EdgeView`.Kiedy chcemy wykorzystać ten obiekt do przetwarzania poza interfejsem biblioteki `networkx`, w celu uniknięcia błędów lepiej go sprowadzić do standardowego typu lub `np.ndarray`.

Graf możemy również za pomocą listy krawędzi

In [ ]:
nx.from_edgelist(
    edgelist=[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4)]
).edges()

Dostęp do listy wierzchołków odbywa się za pomocą metody `nodes`. Tutaj także zwracany jest obiekt networkx `NodeView`

In [ ]:
graph.nodes()

### Podstawowe statystyki grafowe

Liczba wierzchołków

In [ ]:
len(graph)

In [ ]:
graph.number_of_nodes()

Liczba krawędzi

In [ ]:
graph.number_of_edges()

Stopnie wierzchołków

In [ ]:
graph.degree()

### Operacje na "słowniku" w networkx

`Networkx` pod spodem wykorzystuję strukturą opartą o słowniki, co umożliwia  wspiera wiele operacji w podobny sposób

Za pomocą standardowego indeksowania mamy dostęp do sąsiadów danego wierzchołka

In [ ]:
graph[1]

Widok wyświetla również atrybuty wyrzchołków, w przypadku tego grafu nie ma atrybutów, dlatego zwracane są puste słowniki. W celu wyświetlenia samej listy sąsiadów możemy wykorzystać metodę `neighbors`. Przy korzystaniu z tej metody musimy wcześniej przemapować to na listę.

In [ ]:
list(graph.neighbors(1))

Wykorzystanie dwupoziomowego indeksu zwraca attrybuty danego wierzchołka

In [ ]:
graph[1][2]

Sprawdzenie czy wierzchołek występuje w grafie

In [ ]:
4 in graph

In [ ]:
5 in graph

### Inne typy grafów

`nx.Graph` nie jest jedynem typem grafu wspieranym przez bibliotekę `networkx`. Główną różnicą jest wspierany typ krawędzi, rozróżnia się skierowanie krawędzie (posiadają kierunek) oraz równoległe krawezie (powtórzone krawędzie - przydatne w przypadku grafów temporalnych). Porównania typów zawrto w poniższej tabeli.

| Typ | Skierowane krawędzie | Równoległe Krawędzie |
| --- | --- | -- | 
| `nx.Graph` | - | - | 
| `nx.DiGraph` | + | - |
| `nx.MultiGraph` | - | + |
| `nx.MultiDiGraph` | + | + |

Skierowanie krawędzi w grafie możemy zmienić za pomocą metody `to_directed()` lub `to_undirected()`

In [ ]:
directed_graph = graph.to_directed()

In [ ]:
directed_graph.edges()

## Rysowanie grafów

Do wyrysowania grafu możemy wykorzystać interfejs `nx.draw`

In [ ]:
nx.draw(graph)

Niestety domyślny interfejs nie pozwala zaobserować dużo, dlatego autorzy rekomendują wykorzystanie bibliotek dedykowanych do rysowania grafów takich jak `Cytoscape`, `Gephi` czy `Graphviz`.

### Interaktywne wykresy z wykrozystaniem plotly

My wykorzystamy inną ścieżkę, mianowicie wykorzystamy plotly. Materiał przygotowany na bazie tutorialu https://plotly.com/python/network-graphs/. Niestety plotly nie oferuje gotowych interfejsów do wizualizacji grafów, także musimy przejść ręcznie przez ich utworzenie.

Pierwszym wymaganym elementem jest pozycjonowanie wierzchołków w grafie. Do tego możemy wykorzystać gotowe funkcje znajdujące się w module `networkx.drawing.layout`. 

Do interaktywnych wykresów wykorzystamy graf z większą liczbą wierzchołków i krawędzi w tym celu wykorzystamy generator.

In [ ]:
graph = nx.fast_gnp_random_graph(n=50, p=0.03)

In [ ]:
nx.draw(graph, pos=nx.drawing.layout.shell_layout(graph))

In [ ]:
positions = nx.drawing.layout.shell_layout(graph)

Następnie mając pozycje wyrysujmy najpierw wierzchołki z wykorzystaniem metody `go.Scatter`.

In [ ]:
import plotly.graph_objects as go

node_x = []
node_y = []

for node in graph:
    x, y = positions[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    mode="markers",
    hoverinfo="text",
    text=list(graph.nodes()),
    marker=dict(
        size=10,
        line_width=2,
    ),
)

Możemy teraz wyświetlić obecny stan naszego wykresu.


In [ ]:
fig = go.Figure(node_trace)
fig.show()

Teraz dodajmy krawędzie

In [ ]:
edge_x = []
edge_y = []

for edge in graph.edges():
    x0, y0 = positions[edge[0]]
    x1, y1 = positions[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_y.append(y0)
    edge_y.append(y1)
    
edge_trace = go.Scatter(
    x=edge_x,
    y=edge_y,
    line=dict(width=0.5, color="#888"), # Dodane w celu poprawy czytelności
    hoverinfo="none",
    mode="lines",
    showlegend=False
)

In [ ]:
fig = go.Figure([edge_trace, node_trace])
fig.show()

Teraz wprowadźmy jeszcze kilka poprawek usprawniającyh czytelność wykresu

In [ ]:
fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title="Interaktywny wykres",
        titlefont_size=16,
        showlegend=False,
        hovermode="closest",
        annotations=[
            dict(
                text="",
                showarrow=False,
                xref="paper",
                yref="paper",
                x=0.005,
                y=-0.002,
            )
        ],
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    ),
)

fig.show()

Możemy oznaczyć również grupowanie wierzchołków. Wygenerujmy przykładowe mapowanie.

In [ ]:
import numpy as np

groups = np.array_split(graph.nodes(), 4)

Teraz zmienimy metodę rysowania wierzchołków

In [ ]:
node_traces = []
for group_id, node_group in enumerate(groups):
    node_x = []
    node_y = []
    for node in node_group:
        x, y = positions[node]
        node_x.append(x)
        node_y.append(y)

    node_traces.append(
        go.Scatter(
            x=node_x,
            y=node_y,
            mode="markers",
            hoverinfo="text",
            text=node_group,
            name=group_id,
            marker=dict(
                size=10,
                line_width=2,
            ),
            legendgroup=f"{group_id}",
        
        )
    )

In [ ]:
fig = go.Figure(node_traces)
fig.show()

In [ ]:
fig = go.Figure(
    data=[edge_trace, *node_traces],
    layout=go.Layout(
        title="Interaktywny wykres z wyróżnieniem grup",
        titlefont_size=16,
        showlegend=True,
        hovermode="closest",
        annotations=[
            dict(
                text="",
                showarrow=False,
                xref="paper",
                yref="paper",
                x=0.005,
                y=-0.002,
            )
        ],
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        legend_title_text="Group"
    ),
)

fig.update_layout(legend_itemclick=False)
fig.show()